# Preliminary Working Memory Model

This is the model that our team created during the 2024 Neuromatch Summer School.


(Insert a ... "heck" ton of README-style explanation for the preliminary model OR put the info in the main README file for both models.)

## Setup and dependencies

In [1]:
!pip install nilearn --quiet

import os
import re
import tarfile
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

## Figure settings

In [2]:
# will be replaced by the xkcd style

## Parameters and Data Download
The data used for the preliminary model was shared by Neuromatch in the [Project Booklets](https://compneuro.neuromatch.io/projects/fMRI/README.html#:~:text=5%2D23%2C%202021-,HCP%20task%20datasets,-%23) and most of our data preparation is similar to what they have shared in the ```load_hcp_task_with_behaviour.ipynb``` in the [HCP 2021 + behavior](https://compneuro.neuromatch.io/projects/fMRI/README.html#:~:text=View-,HCP%202021%20%2B%20behavior,-HCP%202021) section.

Our target experiments (```TargetExperiments``` variable below) are Working Memory, Emotion, and Language tasks.

In [3]:
N_SUBJECTS = 100
N_PARCELS = 360 # Data aggregated into ROIs from Glasser parcellation
TR = 0.72  # Time resolution, in seconds
HEMIS = ["Right", "Left"]
RUNS   = ["LR","RL"]
N_RUNS = 2

EXPERIMENTS = {
    'MOTOR'      : {'cond':['lf','rf','lh','rh','t','cue']},
    'WM'         : {'cond':['0bk_body','0bk_faces','0bk_places','0bk_tools',
                            '2bk_body','2bk_faces','2bk_places','2bk_tools']},
    'EMOTION'    : {'cond':['fear','neut']},
    'GAMBLING'   : {'cond':['loss','win']},
    'LANGUAGE'   : {'cond':['math','story']},
    'RELATIONAL' : {'cond':['match','relation']},
    'SOCIAL'     : {'cond':['ment','rnd']}
}

TargetExperiments = ['WM', 'EMOTTION', 'LANGUAGE']

TargetConditions  = ["0bk_body", "0bk_faces", "0bk_places", "0bk_tools",
                     "2bk_body", "2bk_faces", "2bk_places", "2bk_tools",
                     "fear"    , "neut"     , "math"      , "story"    ]

In [4]:
fname = "hcp_task.tgz"
url = "https://osf.io/2y3fw/download"

if not os.path.isfile(fname):
  try:
    r = requests.get(url)
  except requests.ConnectionError:
    print("Download FAILED: Connection Error!")
  else:
    if r.status_code != requests.codes.ok:
      print("Download FAILED!")
    else:
      with open(fname, "wb") as fid:
        fid.write(r.content)


HCP_DIR = "./hcp_task"

with tarfile.open(fname) as tfile:
  tfile.extractall('.')

SubjectIDs = np.loadtxt(os.path.join(HCP_DIR, 'subjects_list.txt'), dtype='str')
SubjectIDs = list(SubjectIDs)

C:\Users\User\AppData\Local\Temp\ipykernel_17536\3387722393.py:20: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tfile.extractall('.')


### ```regions.npy``` file and parcels
(Insert doc about what the regions file is)

In [5]:
regions = np.load(f"{HCP_DIR}/regions.npy").T

region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    hemi=['Right']*int(N_PARCELS/2) + ['Left']*int(N_PARCELS/2),
)

In [6]:
"""
THIS SECTION OF CODE WILL BE REPLACED BY CODE THAT DIRECTLY EXTRACTS
THE NUMBERS AND CONSTUCTS THE DICTIONARY FROM THE regions.npy FILE.
"""

ventral_attention_parcels    = [121, 134]

orbital_affective_parcels    = [111, 165, 289, 291, 345]

dorsal_attention_parcels     = [26, 139, 140, 206, 207, 320]

limbic_parcels               = [109, 111, 165, 289, 291, 345]

auditory_parcels             = [23, 102, 103, 123, 172, 173, 
                                174, 282, 286, 287, 288, 303, 352, 353, 354]

default_mode_parcels         = [11, 24, 25, 27, 
                                73, 74, 78, 80, 122, 124, 127, 128, 138, 171, 
                                191, 205, 254, 258, 302, 304, 308, 318, 351]

language_parcels             = [10, 45, 49, 94, 
                                95, 115, 126, 135, 136, 142, 145, 225, 229, 
                                274, 275, 295, 296, 306, 315, 316, 322, 325]

frontoparietal_parcels       = [13, 14, 
                                28, 62, 72, 76, 79, 81, 82, 84, 96, 97, 110, 
                                132, 143, 144, 148, 169, 170, 208, 242, 252,
                                256, 259, 260, 262, 264, 276, 277, 290, 298]

somatomotor_parcels          = [7, 8, 35, 38, 39, 40, 41, 46, 
                                50, 51, 52, 53, 54, 55, 99, 100, 101, 167, 
                                187, 188, 215, 218, 219, 220, 221, 226, 230, 
                                231, 232, 233, 234, 235, 279, 280, 281, 347]

cingulo_opercular_parcels    = [9, 36, 37, 42, 43, 44, 56, 57, 58, 59, 98, 
                                104, 105, 107, 112, 113, 114, 116, 178, 179, 
                                189, 190, 204, 216, 217, 222, 223, 224, 236, 
                                237, 238, 239, 257, 261, 263, 265, 275, 277, 
                                285, 292, 293, 346, 348, 357, 358, 359]

visual_parcels               = [0, 1, 2, 3, 4, 5, 6,
                                12, 15, 16, 17, 18, 19, 20, 21, 22, 47, 48,
                                137, 141, 151, 152, 153, 155, 156, 157, 158, 
                                159, 162, 186, 192, 195, 196, 197, 198, 199, 
                                200, 201, 202, 227, 228, 317, 321, 331, 332, 
                                333, 335, 336, 337, 338, 339, 342]

posterior_multimodal_parcels = [29, 30, 31, 32, 33, 34, 60, 61, 63, 64, 65, 
                                66, 67, 68, 69, 70, 71, 75, 86, 87, 88, 89, 
                                117, 118, 119, 130, 131, 133, 154, 160, 161, 
                                163, 164, 175, 176, 177, 180, 181, 182, 183, 
                                184, 185, 192, 193, 194, 195, 196, 197, 198, 
                                199, 200, 201, 202, 214, 215, 216, 217, 218, 
                                219, 220, 221, 230, 231, 232, 233, 234, 235, 
                                246, 247, 248, 249, 250, 251, 252, 253, 255, 
                                266, 267, 269, 270, 271, 272, 273, 276, 277, 
                                278, 279, 280, 281, 283, 284, 297, 299, 300, 
                                301, 305, 307, 309, 310, 311, 312, 313, 314, 
                                319, 323, 324, 326, 327, 328, 329, 330, 341, 
                                344, 349, 350]

# Dictionary of subnetworks with no. of parcels and the list of corresponding parcels.
subnetworks = {
    f"visual_nw_{len(visual_parcels)}"                             : visual_parcels             ,
    f"limbic_nw_{len(limbic_parcels)}"                             : limbic_parcels             ,
    f"auditory_nw_{len(auditory_parcels)}"                         : auditory_parcels           ,
    f"language_nw_{len(language_parcels)}"                         : language_parcels           ,
    f"somatomotor_nw_{len(somatomotor_parcels)}"                   : somatomotor_parcels        ,
    f"default_mode_nw_{len(default_mode_parcels)}"                 : default_mode_parcels       ,
    f"frontoparietal_nw_{len(frontoparietal_parcels)}"             : frontoparietal_parcels     ,
    f"dorsal_attention_nw_{len(dorsal_attention_parcels)}"         : dorsal_attention_parcels   ,
    f"cingulo_opercular_nw_{len(cingulo_opercular_parcels)}"       : cingulo_opercular_parcels  ,
    f"orbital_affective_nw_{len(orbital_affective_parcels)}"       : orbital_affective_parcels  ,
    f"ventral_attention_nw_{len(ventral_attention_parcels)}"       : ventral_attention_parcels  ,
    f"posterior_multimodal_nw_{len(posterior_multimodal_parcels)}" : posterior_multimodal_parcels
}

## Helper Functions
Here, we define funtions that we will be using for creating the dataframe and modeling.

### Functions for the data preparation

In [7]:
def load_single_timeseries(subject, experiment, run, remove_mean=True):
    """Load timeseries data for a single subject and single run.

    Arguments:
        subject (str):      subject ID to load
        experiment (str):   Name of experiment
        run (int):          (0 or 1)
        remove_mean (bool): If True, subtract the parcel-wise mean
                            (typically the mean BOLD signal is not of interest)

    Returns
        ts (n_parcel x n_timepoint array): Array of BOLD data values
    
    """
    bold_run  = RUNS[run]
    bold_path = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_{bold_run}"
    bold_file = "data.npy"
    ts_path   = f"{bold_path}/{bold_file}"
    
    if not os.path.exists(ts_path):
        raise FileNotFoundError(f"Timeseries file not found: {ts_path}")
    ts = np.load(ts_path)
    
    if remove_mean:
        ts = ts - ts.mean(axis=1, keepdims=True)
    return ts

In [8]:
def load_evs(subject, experiment, run):
    """Load EVs (explanatory variables) data for one task experiment.

    Arguments:
        subject (str): subject ID to load
        experiment (str): Name of experiment
        run (int): 0 or 1

    Returns:
        evs (list of lists): A list of frames associated with each condition
    
    """
    frames_list = []
    task_key = f"tfMRI_{experiment}_{RUNS[run]}"
    for cond in EXPERIMENTS[experiment]["cond"]:
        ev_file  = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
        ev_array = np.loadtxt(ev_file, ndmin=2, unpack=True)
        ev       = dict(zip(["onset", "duration", "amplitude"], ev_array))
        
        # Determine when trial starts, rounded down
        start = np.floor(ev["onset"] / TR).astype(int)
        # Use trial duration to determine how many frames to include for trial
        duration = np.ceil(ev["duration"] / TR).astype(int)
        # Take the range of frames that correspond to this specific trial
        frames = [s + np.arange(0, d) for s, d in zip(start, duration)]
        frames_list.append(frames)

    return frames_list


def load_evs_as_dict(subject, experiment, run):
    """Load EVs (explanatory variables) data for one task experiment.

    Arguments:
        subject (str): subject ID to load
        experiment (str): Name of experiment
        run (int): 0 or 1

    Returns:
        evs (dict): A dictionary of the data associated with each condition
    
    """
    evs = {}
    task_key = f"tfMRI_{experiment}_{RUNS[run]}"

    for cond  in EXPERIMENTS[experiment]["cond"]:
        ev_file = f"{HCP_DIR}/subjects/{subject}/{experiment}/{task_key}/EVs/{cond}.txt"
        if not os.path.exists(ev_file):
            raise FileNotFoundError(f"EV file not found: {ev_file}")
        ev_array  = np.loadtxt(ev_file, ndmin=2, unpack=True)
        evs[cond] = dict(zip(["onset", "duration", "amplitude"], ev_array))

    return evs

In [9]:
def create_dataframe(subject, experiment):
    """
    Creates a dataframe that contains the parcel-based 
    BOLD signals from a subject for each condition.

    Arguments:
        subject (str): subject ID to load
        experiment (str): Name of experiment

    Returns:
        A dataframe of parcel-based BOLD data
        for one subject and one experiment
        
    """
    all_data = []

    for run in range(2): # Run can be 0 (LR) or 1 (RL)
        try:
            ts  = load_single_timeseries(subject, experiment, run)
            evs = load_evs_as_dict(subject, experiment, run)
        except FileNotFoundError as e:
            print(e)
            continue

        n_parcels, n_timepoints = ts.shape

        for condition, ev_data in evs.items():
            onset_times = ev_data["onset"]
            durations   = ev_data["duration"]
            amplitudes  = ev_data["amplitude"]

            for onset, duration, amplitude in zip(onset_times, durations, amplitudes):
                start_frame = int(onset / TR)
                end_frame   = start_frame + int(duration / TR)

                for time_point in range(start_frame, end_frame):
                    if time_point < n_timepoints: # Ensure it is within bounds
                        row = {
                            "sunject"      : subject   ,
                            "experiment"   : experiment,
                            "run"          : RUNS[run] ,
                            "condition"    : condition ,
                            "timepoint"    : time_point,
                            "EV_onset"     : onset     ,
                            "EV_duration"  : duration  ,
                            "EV_amplitude" : amplitude
                        }
                        # Add BOLD signal data for all parcels
                        row.update({f"parcel_{i}": ts[i, time_point] for i in range(n_parcels)})
                        all_data.append(row)

    df = pd.DataFrame(all_data)
    return df

In [10]:
def extract_stats(subject, experiment):
    """Aggregates all data for a subject into
    a dictionary that can be used along with
    "gather_all_subjects_stats()" to create
    the final dataframe.

    Arguments:
        subjects    (list of str): list of SubjectIDs
        experiments (list of str): list of TargetExperiments

    Returns:
        A dictionary of all the data points
        for a subject's specific experiment.

    
    """
    stats_dict = {"subject": subject}
    task_path  = f"{HCP_DIR}/subjects/{subject}/{experiment}/tfMRI_{experiment}_LR/EVs"

    if os.path.exists(task_path):
        for filename in os.listdir(task_path):
            if filename == "Stats.txt":
                filepath = os.path.join(task_path, filename)
                with open(filepath, "r") as file:
                    lines = file.readlines()
                    for line in lines:
                        match = re.match(r"([\w\s-]+): ([\d.]+)", line.strip())
                        if match:
                            key, value = match.groups()
                            stats_dict[f"{experiment}_{key.strip().replace(" ", "_")}"] = float(value)

    return stats_dict

In [11]:
def gather_all_subjects_stats(subjects, experiments):
    """Creates a dataframe containing all data from
    all subjects which stores the parcel-based BOLD signals.

    Arguments:
        subjects    (list of str): list of SubjectIDs
        experiments (list of str): list of TargetExperiments

    Returns:
        A dataframe of parcel-based BOLD data
        for all subjects and all experiments
    
    """
    all_stats = []

    for subject in subjects:
        subject_stats = {"subject": subject}
        for experiment in experiments:
            stats = extract_stats(subject, experiment)
            subject_stats.update(stats)

            # Get the dimensions of DataFrame for this subject and experiment
            try:
                df = create_dataframe(subject, experiment)
                subject_stats[f"{experiment}_num_rows"] = df.shape[0]
                subject_stats[f"{experiment}_num_cols"] = df.shape[1]
            except FileNotFoundError:
                subject_stats[f"{experiment}_num_rows"] = None
                subject_stats[f"{experiment}_num_cols"] = None
        
        all_stats.append(subject_stats)

    stats_df = pd.DataFrame(all_stats)
    return stats_df

In [12]:
def save_stats_to_csv(df, filename):
    """Saves the input dataframe as a csv in working directory.

    Arguments:
        df (dataframe)
        filename (str)
    """
    df.to_csv(filename, index=False)

### Functions related to EDA, correlation, and subnetworks

In [13]:
#
# THIS FUNCTION MIGHT NOT BE NEEDED.
#

def compute_binary_covariance_matrix(cov_matrix, threshold = 0.5):
    """Calculates the binary covariance matrix.

    Arguments:
        cov_matrix (): ???
        threshold  (): ??? (Default is 0.5)

    Returns:
        ??? (Seems to be a 0 or 1 output based on the threshold)
        
    """
    binary_covariance_matrix = (cov_matrix.abs() > threshold).astype (int)
    return binary_covariance_matrix

In [14]:
#
# THIS FUNCTION MIGHT NOT BE NEEDED.
# 
# ALSO, WHAT IS "data" IN THIS FUNCITON?
# THE VARIABLE ISN'T DEFINED PROPERLY IN THE LATER SECTIONS.
#

def plot_correlation_heatmap(parcels, parcel_name):
    """Plots the correlation matrix between parcels.

    Arguments:
        parcels    (list): List of parcel numbers "parcel_name" network
        parcel_name (str): Name of network

    Returns:
        The correlation heatmap of parcels from selected network

    Example usage:
        plot_correlation_heatmap(language_parcels, "Language Network")
        
        Where "language_parcels" is a list
        of parcels in the "Language Network"
    
    """
    # Extract selected parcels
    # "+1" to adjust for the "condition" column being the first
    selected_data = data.iloc[:, [0]+[i+1 for i in parcels]] 

    # Transpose to put tasks in rows and parcels in columns
    transposed_data = selected_data.iloc[:, 1:].transpose()

    # Fix labels and create correlation matrix
    conditions = data["condition"]
    correlation_matrix_tasks         = transposed_data.corr()
    correlation_matrix_tasks.index   = conditions
    correlation_matrix_tasks.columns = conditions

    plt.figure(figsize = (10, 8))
    sns.heatmap(correlation_matrix_tasks, cmap = "coolwarm", 
                center = 0, annot = True, fmt = ".2f")
    plt.title(f"Correlation Matrix Heatmap of Tasks based on BOLD Signals ({parcel_name})")
    plt.show()

In [15]:
#
# THIS FUNCTION MIGHT NOT BE NEEDED.
# 
# ALSO, WHAT IS "result" IN THIS FUNCITON?
# THE VARIABLE ISN'T DEFINED PROPERLY IN THE LATER SECTIONS.
#

def plot_matrices(result, condition_name):
    """Plots the correlation matrix of
    a subnetwork for given condition.
    
    Arguments:
        result         (???): ???
        condition_name (str): Condition of HCP task

    Returns:
        Correlation matrix of given condition across parcels
    
    """
    corr_matrix = result.iloc[:, 1:].corr()
    
    plt.figure(figsize = (12, 10))
    sns.heatmap(corr_matrix, annot = True, cmap = "coolwarm", center = 0)
    plt.title(f"Correlation Matrix of Subnetworks ({condition_name})")
    plt.show()

In [16]:
def plot_bar_chart(filtered_data, condition_name):
    """Creates barchart of mean BOLD signal of subnetwork.

    Arguments:
        filtered_data  (???): ???
        condition_name (str): Subtask of HCP (math, story, etc.)

    Returns:
        Bar plot of mean BOLD signal for given condition in subnetwork.
    """
    mean_values = filtered_data.iloc[:, 1:].mean()
    
    mean_values.plot(kind = "bar", figsize = (12, 6))

    plt.title(f"Mean BOLD Signal of Subnetworks ({condition_name})")
    plt.ylabel("Mean BOLD Signal")
    plt.xlabel("Subnetwork")
    plt.show()

In [17]:
def calculate_mean_sem(data, selected_networks, condition_name):
    """
    ?????
    """
    condition_data = data[data["condition"].str.startswith(condition_name)]

    mean_values = condition_data[selected_networks].mean()
    sem_values  = condition_data[selected_networks].sem()

    return mean_values, sem_values, len(condition_data)

In [18]:
def plot_combined_bar_chart(data, selected_networks, conditions, color_palette):
    """Generates combined bar chart of mean BOLD signals
    of selected subnetworks for different conditions.
    
    !!! Uses the "calculate_mean_sem()" function. !!!

    Arguments:
        data              (????): Filtered data for a subnetwork
        selected_networks (list): List of desired subnetworks
        conditions        (list): Selected HCP subtasks (math, story, etc.)
        color_pallette    (list): List of colors for each condition

    Returns:
        Plot of mean BOLD signal for selected subnetworks by condition
    
    """
    fig, ax   = plt.subplots(figsize = (15, 8))
    bar_width = 0.15

    # Position of bars on x-axis
    r = np.arange(len(selected_networks))

    for i, condition in enumerate(conditions):
        mean_values, sem_values, num_rows = calculate_mean_sem(data, selected_networks, condition)

        if num_rows > 1 and (mean_values.isnull().any() or sem_values.isnull().any()):
            print(f"Skipping condition {condition} due to NaN values in mean or SEM.")
            continue
        if num_rows == 1:
            ax.bar(r + i * bar_width, mean_values,
                   width = bar_width, label = condition,
                   color = color_palette[i])
        else:
            ax.bar(r + i * bar_width, mean_values,
                   yerr  = sem_values, label = condition,
                   color = color_palette[i])

        ax.set_xlabel("Subnetwork")
        ax.set_ylabel("Mean BOLD Signal")
        ax.set_title("Mean BOLD Signals of Selected Subnetworks\nunder Different Conditions")

        ax.set_xticks(r + bar_width * (len(conditions) - 1) / 2)
        ax.set_xticklabels(selected_networks, rotation = 45)

        ax.legend()

        plt.show()

### Functions for the matrix construction and modeling

## Creating dataframes
Here, we create dataframes that contain the data relative to subjects and ROIs (parcels).

In the preliminary model, this datapoints are the average BOLD signals for each parcel.

In [ ]:
# Using functions that we have already defined ...
all_stats_df = gather_all_subjects_stats(SubjectIDs, TargetExperiments)

output_file_name = "all_subjects_stats.csv"

save_stats_to_csv(all_stats_df, output_file_name)

data = pd.read_csv("all_subjects_stats.csv")
# Assignment has been fixed but the 2nd line of this cell doesn't run.

## EDA

### Parcels, Subnetworks, and Conditions

In [ ]:
# Plot heatmaps for relevant subnetworks
plot_correlation_heatmap(visual_parcels         , "Visual Network"        )
plot_correlation_heatmap(language_parcels       , "Language Network"      )
plot_correlation_heatmap(default_mode_parcels   , "Default Mode Network"  )
plot_correlation_heatmap(frontoparietal_parcels , "Frontoparietal Network")

In [ ]:
# Find all parcels in the data
all_parcels = set([int(col.split("_")[1]) for col in data.columns if col.startswith("parcel_")])

# Find parcels that are not in any network
non_nw_parcels = all_parcels - set(sum(subnetworks.values(), []))

In [ ]:
#
# ARE THE FOLLOWING LINES OF CODE USED IN THE PRELIMINARY MODEL?
#

# Calculate the average for each subnetwork
for nw, parcels in subnetwork.items():
    columns  = [f"parcel_{p}" for p in parcels if f"parcel_{p}" in data.columns]
    data[nw] = data[columns].mean(axis = 1)

# Calculate the average for non-network parcels
columns = [f"parcels_{p}" for p in non_nw_parcels if f"parcels_{p}" in data.columns]

data[f"non_nw_{len(non_nw_parcels)}"] = data[columns].mean(axis = 1)

# Select the condition and new subnetwork columns
result = data[["condition"] + list(subnetworks.keys()) + [f"non_nw_{len(non_nw_parcels)}"]]

# Save to new CSV file in the working directory
result.to_csv("subnetworks_data.csv", index = False)

# Check the results
result.head()

#### Covariance and correlation matrix looking at subnetworks
(Insert the goal of this section of code!)

(Although, this might be redundant because all meaningful data is lost when we average all BOLD and then look at the correlation.)

In [ ]:
cov_matrix = result.iloc[:, 1:].cov()

plt.figure(figsize = (12, 10))
sns.heatmap(cov_matrix, annot = True, cmap = "coolwarm", center = 0)
plt.title("Covariance Matrix of Subnetworks")
plt.show()

In [ ]:
corr_matrix = result.iloc[:, 1:].corr()

plt.figure(figsize = (12, 10))
sns.heatmap(corr_matrix, annot = True, cmap = "coolwarm", center = 0)
plt.title("Correlation Matrix of Subnetworks")
plt.show()

In [ ]:
# Define combined conditions:
combined_conditions = {
    "0bk"      : result[result["condition"].str.startswith("0bk")],
    "2bk"      : result[result["condition"].str.startswith("2bk")],
    "emotion"  : result[result["condition"].isin(["fear","neut"])],
    "language" : result[result["condition"].isin(["math","story"])]
}

# Plot the matrices for the combined conditions
for condition_name, result in combined_conditions.items():
    plot_matrices(result, condition_name)

In [ ]:
# Plot bar charts for each condition
# Plotting mean BOLD signals for each subnetwork and condition
conditions = ["0bk", "2bk", "fear", "neut", "math", "story"]

for condition in conditions:
    filtered_data = result[result["condition"].str.startswith(condition)]
    plot_bar_chart(filtered_data, condition)

In [ ]:
# Plot the combined bar chart for all conditions

# Load the data
subnetworks_data = pd.read_csv("subnetworks_data.csv")

# Select relevant networks
selected_networks = ["visual_nw_52", "frontoparietal_nw_31",
                     "default_mode_nw_23", "language_nw_22",
                     "non_nw_36"]

# Filter dataset
filtered_data = subnetworks_data[["condition"] + selected_networks]

# Conditions the same as the cell above
conditions = ["0bk", "2bk", "fear", "neut", "math", "story"]
# Color palette:
color_palette = ["orange", "red", "black", "grey", "cyan", "blue"]

# Plot the combined bar chart
plot_combined_bar_chart(filtered_data, selected_networks, conditions, color_palette)

### PCA

In [ ]:
#
# WHICH CELLS FROM THE "PCA(EDA)" SECTION ARE NEEDED HERE?
#

## Matrix Construction for the Model

In [ ]:
##########################################

## The MLP Model (2 output version)

### Test and Train data

### Model Setup

### Taining

### Model Summary

### Evaluation and Training History

### Using the Model to predict Emotion and Language tasks
After our Working Memory Model was trained, we gave it the Emotion and Language task data.

### Comparison of Model Predictions
Here, we used 3D plots to compare how our model performed in terms of predicting the different conditions of Emotion and Language tasks.